# Test for issues with concurrent chats

In [1]:
# setup code
import dotenv

dotenv.load_dotenv("../../.env")

import nest_asyncio

nest_asyncio.apply()

## Test Single chat at a time

In [2]:
from llama_index.llms import OpenAI, ChatMessage, MessageRole, ChatResponseAsyncGen


# Necessary to use the latest OpenAI models that support function calling API
async def run_chat():
    llm = OpenAI(model="gpt-3.5-turbo-0613")

    streaming_chat_response: ChatResponseAsyncGen = await llm.astream_chat(
        [ChatMessage(content="Hi", role=MessageRole.USER)]
    )
    response_str = ""
    async for chat_response in streaming_chat_response:
        response_str = chat_response.message.content

    print("final response_str: " + response_str)


await run_chat()

final response_str: Hello! How can I assist you today?


In [3]:
# Test basic concurrency
from typing import Callable
from concurrent.futures import ThreadPoolExecutor
import asyncio


def run_in_event_loop(func: Callable):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(func())
    loop.close()


async def run_task_n_times(task: Callable, n: int):
    def thread_task():
        run_in_event_loop(task)

    # Run two calls to run_chat concurrently and wait for them to finish
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(thread_task) for _ in range(n)]
        for future in futures:
            future.result()

In [4]:
await run_task_n_times(run_chat, 5)

final response_str: Hello! How can I assist you today?
final response_str: Hello! How can I assist you today?
final response_str: Hello! How can I assist you today?
final response_str: Hello! How can I assist you today?
final response_str: Hello! How can I assist you today?


## Testing with `OpenAIAgent` based chat engine

The above output looks fine for concurrent cases. Indicates there is no issue with concurrent chats coming from the LLM layer.
Now will try from the other end of the stack with a full `OpenAIAgent` chat_engine.

In [5]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI
from llama_index.agent.openai_agent import StreamingAgentChatResponse


async def run_chat():
    # initialize llm
    llm = OpenAI(model="gpt-3.5-turbo-0613")

    # initialize openai agent
    agent = OpenAIAgent.from_tools([], llm=llm, verbose=True)
    streaming_chat_response: StreamingAgentChatResponse = await agent.astream_chat("Hi")
    response_str = ""
    for text in streaming_chat_response.response_gen:
        response_str += text
    print("final response_str: " + response_str)

Sanity check that the `OpenAIAgent` chat engine works on its own (no concurrent chats)

In [6]:
await run_chat()

final response_str: Hello! How can I assist you today?


Ok now check for concurrency

In [7]:
await run_task_n_times(run_chat, 5)

final response_str: Hello! How can I assist you today?
final response_str: Hello! How can I assist you today?
final response_str: Hello! How can I assist you today?
final response_str: Hello! How can I assist you today?
final response_str: Hello! How can I assist you today?
